In [6]:

%load_ext autoreload 
# %aimport rl_envs.grid_world_env

%autoreload 2
import torch
import math
from torch.utils.tensorboard import SummaryWriter # type: ignore

# from agents.A2C import A2CAgent
from agents.A2C_cliff import A2CAgent
from tools.helper import *
import  gymnasium  as gym
from rl_envs.new_gym_grid_world_env import GridWorldEnv
from torch.nn import functional as F
from collections import defaultdict
import itertools
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
LEARN_RATE = 1e-2
DISCOUNTED_FACTOR = 0.99

FORBIDDEN_REWARD = -10
HITWALL_REWARD = -10
TARGET_REWARD = 1

SEED = 666

In [8]:
# env = GridWorldEnv(size=3,fixed_map = True, seed=SEED, forbidden_grids=[(1,1)], target_grids=[(2,2)], forbidden_reward=FORBIDDEN_REWARD, hit_wall_reward=HITWALL_REWARD, target_reward=TARGET_REWARD)
# env = GridWorldEnv(fixed_map = True, forbidden_grids=[(1,1),(1,2), (2,2),(3,1),(3,3),(4,1)], target_grids=[(3,2)], forbidden_reward=FORBIDDEN_REWARD, hit_wall_reward=HITWALL_REWARD, target_reward=TARGET_REWARD)

# env = gym.make("CliffWalking-v0")
# torch.autograd.set_detect_anomaly(False, check_nan=True)
# env.seed(args.seed)
torch.manual_seed(SEED)

In [9]:
episode_rewards = []
episode_lengths = []


In [10]:
env = gym.make("CliffWalking-v0")
agent = A2CAgent(int(env.observation_space.n), int(env.action_space.n),  discounted_factor=0.99999,  save_action=True)

# env = GridWorldEnv(fixed_map = True, forbidden_grids=[(1,1),(1,2), (2,2),(3,1),(3,3),(4,1)], target_grids=[(3,2)], forbidden_reward=FORBIDDEN_REWARD, hit_wall_reward=HITWALL_REWARD, target_reward=TARGET_REWARD)
# agent = A2CAgent(2, 5, height= env.height, width=env.width, discounted_factor=DISCOUNTED_FACTOR)

# env = gym.make("CartPole-v1")
# agent = A2CAgent(4, 2, lr = LEARN_RATE, discounted_factor=DISCOUNTED_FACTOR)

writer = SummaryWriter()
num_episodes = 300000
episode_len = 10000
eps = np.finfo(np.float32).eps.item()
# 第一次收集改为随机收集
trajectory = []
obs, _ = env.reset()
# for _ in range(1000):
#     state = tuple(obs['agent'])
#     action = agent.get_behavior_action(state)
#     obs, reward, terminated, truncated, info = env.step(action)
#     trajectory.append((state, action, reward+10))
running_reward = -10
episode_rewards = defaultdict(float)
for i_episode  in range(num_episodes):
    # 首先, 根据 policy 生成 episode
    next_state, _ = env.reset()
    # trap_flag = False
    # 初始策略是不是有比较大的影响? 
    episode_record = []
    del agent.saved_log_probs[:]
    for t in itertools.count():
        # del agent.saved_log_prob
        state = next_state
        action = agent.get_action(state) # action 这里也有随机性
        next_state, reward, terminated, truncated, info = env.step(action)
        episode_record.append((state, action, reward, next_state))
        # Update statistics
        episode_rewards[i_episode] += reward
        if t % 1000 == 0:
            print("\rStep {} @ Episode {}/{} ({}, {})".format(
                    t, i_episode + 1, num_episodes, episode_rewards[i_episode], episode_rewards[i_episode - 1]))
        if terminated or truncated or reward == -10:
            break
    discounted_return = 0
    for i, (state, action, reward, next_state) in enumerate(reversed(episode_record)):
        # calculate TD target
        discounted_return = agent.discounted_factor * discounted_return + reward
        
        v_value = agent.value_net(state)
        # value_next = agent.value_net(next_state)

        # TD_target = reward + agent.discounted_factor * value_next
        advantage = discounted_return - v_value

        # updates
        agent.optimizer_v.zero_grad()
        loss2 =  F.mse_loss(torch.tensor(discounted_return), v_value.squeeze())
        loss2.sum().backward(retain_graph=True)
        agent.optimizer_v.step()

        # action_probs = agent.policy_net(state)        
        # action_prob = action_probs[action]
        action_prob = agent.saved_log_probs[i]

        agent.optimizer.zero_grad()
        # loss = - torch.log(action_prob) * advantage
        loss = - action_prob * advantage
        loss.sum().backward()
        # torch.nn.utils.clip_grad.clip_grad_norm_(agent.policy_net.parameters(), 100)
        agent.optimizer.step()



Step 0 @ Episode 1/300000 (-100.0, 0.0)
Step 1000 @ Episode 1/300000 (-9317.0, 0.0)
Step 2000 @ Episode 1/300000 (-19128.0, 0.0)
Step 3000 @ Episode 1/300000 (-29434.0, 0.0)
Step 4000 @ Episode 1/300000 (-40136.0, 0.0)
Step 5000 @ Episode 1/300000 (-51927.0, 0.0)
Step 6000 @ Episode 1/300000 (-59956.0, 0.0)
Step 7000 @ Episode 1/300000 (-69371.0, 0.0)
Step 8000 @ Episode 1/300000 (-79677.0, 0.0)
Step 9000 @ Episode 1/300000 (-88795.0, 0.0)
Step 10000 @ Episode 1/300000 (-99002.0, 0.0)
Step 0 @ Episode 2/300000 (-1.0, -107005.0)
Step 1000 @ Episode 2/300000 (-11792.0, -107005.0)
Step 2000 @ Episode 2/300000 (-20415.0, -107005.0)
Step 3000 @ Episode 2/300000 (-33889.0, -107005.0)
Step 4000 @ Episode 2/300000 (-45878.0, -107005.0)
Step 5000 @ Episode 2/300000 (-59154.0, -107005.0)


KeyboardInterrupt: 

In [ ]:
# """
# output 
# """
# action_mappings = ["↑", "→", "↓", "←"]
# for i in range(48):
#     if i % 12 == 0:
#         print()
#     action = agent.get_action(i)
#     print(action_mappings[action], end=" ")

In [ ]:
# # visualize_in_gym(agent, "CartPole-v1")
policy = agent.generate_policy_table(env.height, env.width)
print(env)
print_by_dict(env, policy)

for i in range(env.height):
    print("[", end=" ")
    for j in range(env.width):
        state = (i,j)
        action = np.argmax(policy[state])
        print(env.action_mappings[action], end=" ")
    print("]")

[ 0.00 0.00 0.00 0.00 0.00 ]
[ 0.00 -10.00 -10.00 0.00 0.00 ]
[ 0.00 0.00 -10.00 0.00 0.00 ]
[ 0.00 -10.00 1.00 -10.00 0.00 ]
[ 0.00 -10.00 0.00 0.00 0.00 ]

[ [[0.05448444 0.7002051  0.00596851 0.00402115 0.23532088]] [[0.00799906 0.81794536 0.00550334 0.02788417 0.14066802]] [[0.0067288  0.7907838  0.00520222 0.04675211 0.15053302]] [[0.5114877  0.22043052 0.00483145 0.04819528 0.21505506]] [[0.7650036  0.011203   0.00682375 0.09995566 0.11701391]] ]
[ [[0.1994018  0.00824627 0.6274428  0.00681459 0.15809461]] [[0.0727447  0.0526539  0.5979391  0.22148496 0.05517739]] [[0.03413967 0.7674575  0.15028743 0.01947347 0.02864189]] [[0.3349588  0.4190122  0.04481204 0.00540841 0.19580865]] [[0.73728496 0.00667041 0.05386091 0.08127114 0.12091255]] ]
[ [[0.10492554 0.06758191 0.62568295 0.00526727 0.19654228]] [[0.01317008 0.01543762 0.01304096 0.8232991  0.13505217]] [[0.88904977 0.0488671  0.02026117 0.02207359 0.01974836]] [[0.00673974 0.86289847 0.05750281 0.00341097 0.06944805]] [[0.78

d:\codes\RL_playground\agents\A2C.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  out = torch.tensor(sq_inp, dtype=torch.int64)


In [ ]:
gridworld_demo(agent, env, repeat_times=500)


reward: 1, distance: [array([4, 1]), array([4, 2]), array([3, 2])]
TERMINATE
reward: 1, distance: [array([0, 3]), array([1, 3]), array([1, 4]), array([2, 4]), array([3, 4]), array([4, 4]), array([4, 3]), array([4, 2]), array([3, 2])]
TERMINATE
reward: 1, distance: [array([3, 4]), array([4, 4]), array([4, 3]), array([4, 2]), array([3, 2])]
TERMINATE
reward: 1, distance: [array([3, 0]), array([2, 0]), array([1, 0]), array([0, 0]), array([0, 1]), array([0, 2]), array([0, 3]), array([1, 3]), array([1, 4]), array([2, 4]), array([3, 4]), array([4, 4]), array([4, 3]), array([4, 2]), array([3, 2])]
TERMINATE
reward: 1, distance: [array([2, 2]), array([3, 2])]
TERMINATE
reward: 1, distance: [array([3, 1]), array([3, 2])]
TERMINATE
reward: 1, distance: [array([1, 2]), array([1, 3]), array([1, 4]), array([2, 4]), array([3, 4]), array([4, 4]), array([4, 3]), array([4, 2]), array([3, 2])]
TERMINATE
reward: 1, distance: [array([4, 2]), array([3, 2])]
TERMINATE
reward: 1, distance: [array([3, 3]), ar

KeyboardInterrupt: 

In [ ]:
# env.max_steps = 10
# gridworld_demo(agent, env, repeat_times=500)
# gridworld_demo(agent, forbidden_reward=FORBIDDEN_REWARD, hit_wall_reward=HITWALL_REWARD, target_reward=TARGET_REWARD)
# visualize_in_gym(agent, "CartPole-v1")
# visualize_in_gym(agent, "CliffWalking-v0")


IndexError: tuple index out of range